In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import spacy
from langdetect import detect

# Cargar modelo BERT solo una vez
model = SentenceTransformer('distiluse-base-multilingual-cased')

# Cargar modelos de spaCy para múltiples idiomas
nlp_models = {
    'es': spacy.load('es_core_news_md'),
    'en': spacy.load('en_core_web_md'),
    'fr': spacy.load('fr_core_news_md')
}

def detectar_idioma(texto):
    return detect(texto)

def cargar_modelo_spacy(idioma):
    return nlp_models.get(idioma, nlp_models['en'])

def extraer_frases_clave(texto, nlp):
    doc = nlp(texto)
    oraciones = [sent.text.strip() for sent in doc.sents]
    return oraciones

def procesar_y_guardar_articulos(input_excel, output_excel):
    # Cargar los artículos del archivo Excel
    df = pd.read_excel(input_excel)
    
    # Crear un diccionario para los modelos ya cargados
    modelos_cargados = {}

    # Lista para guardar los resultados
    resultados = []

    # Procesar cada artículo
    for _, row in df.iterrows():
        texto = row['Texto']
        id_articulo = row['IDArticulo']
        
        # Detectar idioma y cargar el modelo spaCy correspondiente (si no está en cache)
        idioma = detectar_idioma(texto)
        if idioma not in modelos_cargados:
            modelos_cargados[idioma] = cargar_modelo_spacy(idioma)
        
        # Extraer frases clave usando el modelo correspondiente
        frases_clave = extraer_frases_clave(texto, modelos_cargados[idioma])

        # Guardar cada frase con su artículo ID
        resultados.extend({'IDArticulo': id_articulo, 'Frase': frase} for frase in frases_clave)

    # Guardar los resultados en un archivo Excel
    pd.DataFrame(resultados).to_excel(output_excel, index=False)

# Ejemplo de uso:
procesar_y_guardar_articulos('ArticulosOriginales.xlsx', 'FrasesArticulos.xlsx')